<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/Conversion_PyTorch_to_Caffe2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install onnx

help(torch.onnx.export)


In [0]:
#!pip install --upgrade --force-reinstall torch


In [0]:
#!pip install torchvision

#!pip uninstall torchvision
#!pip install -c pytorch pytorch-nightly torchvision cudatoolkit=10.0
!pip install torchvision

In [0]:
# Some standard imports
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torchvision

# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init

### Define the paths

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
checkpoint_name = 'akash-mobilenet_v2-FER1-60perc.pt'
onnx_export_path = base_path + "ONNX/akash_mobilenet_60_bc2.onnx"

To export a model, you call the torch.onnx._export() function. 
This will execute the model, recording a trace of what operators are used to compute the outputs. 
Because _export runs the model, we need provide an input tensor x. 
The values in this tensor are not important; it can be an image or a 
random tensor as long as it is the right size.

In [0]:

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.


# A model class instance (class not shown)
torch_model = torchvision.models.mobilenet_v2(pretrained=False)
torch_model.classifier[1] = torch.nn.Linear(1280, 7)

#print(model)

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None

# Load the weights from a file (.pth usually)
state_dict = torch.load(base_path + checkpoint_name, map_location=torch.device('cpu'))

# Load the weights now into a model net architecture defined by our class
torch_model.load_state_dict(state_dict)

# set the train mode to false since we will only run the forward pass.
torch_model.train(False)



In [0]:
# Create the right input shape (e.g. for an image)
dummy_input = torch.randn(1, 3, 256, 256)


torch.onnx.export(torch_model, dummy_input, onnx_export_path)

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               dummy_input,             # model input (or a tuple for multiple inputs)
                               export_path,             # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file

torch_out is the output after executing the model. Normally you can ignore this output, but here we will use it to verify that the model we exported computes the same values when run in Caffe2.

Now let’s take the ONNX representation and use it in Caffe2. This part can normally be done in a separate process or on another machine, but we will continue in the same process so that we can verify that Caffe2 and PyTorch are computing the same value for the network:

In [0]:
import onnx
import caffe2.python.onnx.backend as onnx_caffe2_backend

In [27]:
# Load the ONNX ModelProto object. model is a standard Python protobuf object
model = onnx.load(export_path)

# prepare the caffe2 backend for executing the model this converts the ONNX model into a
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be
# availiable soon.
prepared_backend = onnx_caffe2_backend.prepare(model)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph of the model itself contains inputs for all weight parameters, after the input image.
# Since the weights are already embedded, we just need to pass the input image.
# Set the first input.
W = {model.graph.input[0].name: dummy_input.data.numpy()}

# Run the Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

print("Exported model has been executed on Caffe2 backend, and the result looks good!")

Exported model has been executed on Caffe2 backend, and the result looks good!
